In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd 
import numpy as np

import sklearn

import matplotlib.pyplot as plt
# from torchtext.legacy import data
import spacy
# import en_core_web_sm

from src.tdde13.data_handling import * #preprocess, preprocess_text, create_adjacency_matrix, create_label_lookup
from src.tdde13.lstm import LSTM
# , exclude=["ner", "parser"]
nlp = spacy.load("en_core_web_sm", exclude=["ner", "parser"])

import pickle

import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


Error: Session cannot generate requests

In [ ]:
# mit reviews arbeiten
# wer könnten mit nlp gearbeitet haben? John?
# 

In [2]:
df = pd.read_csv('datasets/book_reviews/finalv2.csv')

In [2]:
# next(iter(pd.read_csv('datasets/book_reviews/final.csv', chunksize=10)))
df = pd.read_csv('datasets/book_reviews/finalv2.csv')
# df5000 = df[:5000]
df_subset = df[2000:8000]

load = True
if load:
    with open('preprocessed_reviews_2K-8K.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        desc, vocab, word_to_ix = pickle.load(f)
else:
    desc, vocab, word_to_ix = preprocess_text(nlp, df10000.description)
    with open('objs5000.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([desc, vocab, word_to_ix], f)

label_to_idx = create_label_lookup(df_subset)
clean_desc, labels = remove_empty_descriptions(desc, df_subset.genres)

In [3]:
df_subset = df[2000:8000]
with open('objs10000.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    descriptions, descriptions_vocab, descriptions_word_to_ix = pickle.load(f)

In [4]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',
     categories=categories, shuffle=True, random_state=42)

In [3]:
clean_desc, labels = remove_empty_descriptions(desc, df_subset.genres)

In [24]:
cut_length = 2000
clean_desc = np.array([text[:cut_length] for text in clean_desc], dtype=object)

In [6]:
# df_subset = df[:10000]
clean_desc, labels = remove_empty_descriptions(desc, df_subset.genres)
padded_desc, max_len = pad_sequence(clean_desc)

In [5]:
idx_train, idx_val, idx_test = split(labels=labels, train_size=0.6, val_size=0.2, test_size=0.2)

In [8]:
X_train, X_test, y_train, y_test = get_data()

10000
9892


In [17]:
vectorizer = CountVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [18]:
X_train_transformed.shape

(7913, 42003)

In [ ]:
from src.tdde13.baselines import MLP

input_size = X_train_transformed.shape[1]
output_size = 10

mlp = MLP(input_size, output_size)

mlp.train_mlp(X_train_transformed, X_test_transformed, y_train, y_test)

Error: Session cannot generate requests

In [11]:
X.shape

(7913, 42003)

In [15]:
len(vectorizer.vocabulary_)

42003

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer # we could have use TfidfVectorizer too. From sklearn doc : "tf-idf vectors are also known to work well in practice"
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pipe = Pipeline([('vectorizer', CountVectorizer()), ('nbm', MultinomialNB())])

pipe.fit(X_train, y_train)

preds = pipe.predict(X_test)
print(classification_report(y_test, preds))
print(accuracy_score(y_test, preds))

                                        precision    recall  f1-score   support

                              children       0.00      0.00      0.00         9
                       comics, graphic       0.57      0.08      0.13        53
                   fantasy, paranormal       0.68      0.80      0.73       484
                               fiction       0.66      0.73      0.69       390
history, historical fiction, biography       0.00      0.00      0.00        48
              mystery, thriller, crime       0.96      0.18      0.31       143
                           non-fiction       0.00      0.00      0.00        31
                                poetry       0.00      0.00      0.00         2
                               romance       0.64      0.90      0.75       467
                           young-adult       0.65      0.51      0.57       352

                              accuracy                           0.66      1979
                             macro avg

/home/andreas/anaconda3/envs/tdde13/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pd.DataFrame(labels).iloc[idx_val.tolist(), :].value_counts().plot(kind='bar')

In [11]:
enc_descriptions = torch.tensor([[word_to_ix[word] for word in d] for d in padded_desc])
enc_labels = torch.tensor([label_to_idx[label] for label in labels]).long()


In [41]:
predictions.apply_(lambda x: idx_to_label[x])

TypeError: an integer is required (got type str)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(label, predictions.numpy(), target_names=list(label_to_idx.keys())))

In [ ]:

# what are the predictions of the MLP
# maybe truncate long sentences

prediction_train = lstm(enc_descriptions[idx_train]).argmax(axis=1)
label_train = enc_labels[idx_train]
print(classification_report(label_train, prediction_train.numpy(), target_names=list(label_to_idx.keys())))

In [2]:
# 987 X 1: 231sec
# max_len = 5000

# padded_desc = clean_desc

padded_desc, labels, vocab, word_to_ix, label_to_idx, idx_train, idx_val, idx_test = load_reviews_and_descriptions()
max_len = max([len(x) for x in padded_desc])
from src.tdde13.lstm import LSTM
lstm = LSTM(len(vocab), max_len, use_glove=True)
lstm.train_lstm(padded_desc, labels, vocab, word_to_ix, label_to_idx, idx_train, idx_val, idx_test)


Joining
Splitting
Encode Description
Epoch: 0
